# Retriever and Chain with Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("attention.pdf")
docs = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings(model="mxbai-embed-large"))

In [4]:
query = "An attention function can be described as mapping a query "
result = db.similarity_search(query)

In [5]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")


In [6]:
# Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context. 
    Think step by step before providing a detailed answer. 
    I will tip you $1000 if the user finds the answer helpful. 
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [7]:
# Chain introduction
# Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
retriever = db.as_retriever()

In [17]:
from langchain.chains.retrieval import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retriever_chain.invoke({"input": "Can you give me the conclusion of this research?"})

In [19]:
response['answer']

'Based on the provided context, the conclusion of the research can be summarized as follows:\n\n1. The authors trained two models, a baseline Transformer and a variant with dilated convolutions, on two datasets: the Penn Treebank and a semi-supervised setting combining the Penn Treebank with the larger high-confidence and BerkleyParser corpora.\n2. The authors found that the dilated convolution model performed better than the baseline Transformer in terms of translation quality, while also being more computationally efficient.\n3. The authors attributed the improved performance of the dilated convolution model to its ability to capture longer-range dependencies in the input sequence, which is important for tasks like machine translation.\n4. The authors also found that self-attention mechanisms in the model were useful for interpretability and understanding how different attention heads were learning different aspects of the input sequence.\n5. The authors estimated the number of float